In [1]:
%%javascript

window.load_remote_theme = true
var theme_js = "https://odhk.github.io/hyrule_theme/custom.js";

window.load_local_theme = function(){
    var hostname = document.location.hostname
    return ((hostname == "localhost" || hostname == '127.0.0.1') && !load_remote_theme)
}

var url = load_local_theme() ? document.location.origin + "/files/theme/custom.js" : theme_js

$.getScript(url)

<IPython.core.display.Javascript object>

In [1]:
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
import gc
import lightgbm as lgb
import keras
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import TruncatedSVD
import random
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Lambda, Embedding,constraints,\
Dropout, Activation,GRU,Bidirectional,Subtract, Permute, TimeDistributed, Reshape
from keras.layers import Conv1D,Conv2D,MaxPooling2D,GlobalAveragePooling1D,GlobalMaxPooling1D, MaxPooling1D, Flatten
from keras.layers import CuDNNGRU, CuDNNLSTM, SpatialDropout1D,Layer, initializers, regularizers
from keras.layers.merge import concatenate, Concatenate, Average, Dot, Maximum, Multiply, Subtract
from keras.models import Model
from keras.optimizers import RMSprop,Adam
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import SGD
from keras import backend as K
from sklearn.decomposition import TruncatedSVD, NMF, LatentDirichletAllocation
import gc
import tensorflow as tf
from keras.activations import softmax

from keras.utils import plot_model
from keras.layers import *
from keras.applications.imagenet_utils import _obtain_input_shape
from sklearn.metrics import log_loss
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from scipy.special import erfinv

import gensim
from gensim.models import word2vec
from gensim.models.word2vec import LineSentence
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer

train = pd.read_csv('train_set.csv')
test = pd.read_csv('test_set.csv')

label = train['class']

char_embedding_index = {}
f = open('word_embedding_300dim_new2.txt')
for line in tqdm(f):
    values = line.split()
    word = values[0].lower()
    embeddings = np.asarray(values[1:], dtype = 'float32')
    char_embedding_index[word] = embeddings
f.close()

len(char_embedding_index)

api_embedding_file_path = 'word_embedding_300dim_new2.txt'
embedding_dim = 300
max_nb_api = len(char_embedding_index) + 1
max_sequence_length_api =1000

all_word = list(train['word_seg'].values) + list(test['word_seg'].values)

all_word_list = []
for i in list(all_word):
    all_word_list.append(i.split(' '))

len(all_word_list)

train['word_seg'].apply(lambda x:len(x.split(' '))).describe()

data = pd.DataFrame()

gc.collect()

tokenizer_api = Tokenizer()

tokenizer_api.fit_on_texts(all_word_list)

api_seq = tokenizer_api.texts_to_sequences(all_word_list)

data['word_seq'] = pad_sequences(api_seq, maxlen = max_sequence_length_api, padding='post').tolist()

api_index = tokenizer_api.word_index
print('Found %s unique tokens' % len(api_index))
nb_words = min(max_nb_api, len(char_embedding_index)+1)
api_embedding_matrix = np.zeros((nb_words, embedding_dim))

for word, i in tqdm(api_index.items()):
    embedding_vector = char_embedding_index.get(word)
    if embedding_vector is not None:
        api_embedding_matrix[i] = embedding_vector
    else:
        print('no char%s'%word)

data['id'] = train['id']

train_FE = data.iloc[:len(label),:]
test_FE = data.iloc[len(label):,:]

os.environ["CUDA_VISIBLE_DEVICES"] = "4"

def get_input(df):
    
    _input = [np.array(df.word_seq.values.tolist())]
    
    
    return _input 

/home/libo/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
1271461it [02:14, 9487.33it/s]
  2%|▏         | 23846/1271460 [00:00<00:05, 238391.45it/s]

Found 1271460 unique tokens


100%|██████████| 1271460/1271460 [00:05<00:00, 236701.58it/s]


In [10]:
def RNN1():
    
    #Input layer
    char_input = Input(shape=(max_sequence_length_api,), dtype='int32')
    
    api_embedding_layer = Embedding(nb_words,
        embedding_dim,
        weights=[api_embedding_matrix],
        input_length=max_sequence_length_api,
        trainable=False)
    
    #RNN
    char_rnn_layer_1 = Bidirectional(CuDNNLSTM(150, 
                               return_sequences=True))
    char_rnn_layer_2 = Bidirectional(CuDNNLSTM(150, 
                               return_sequences=True))
    
    char_embedding = api_embedding_layer(char_input)
    #char_embedding = SpatialDropout1D(0.2)(char_embedding)
    
    #双层BiLSTM
    char_rnn_1= char_rnn_layer_1(char_embedding)
    char_rnn_2= char_rnn_layer_2(char_rnn_1)
    
    #max avg
    char_att_maxpooling_1 = GlobalMaxPooling1D()(char_rnn_1)
    char_att_avgpooling_1 = GlobalAveragePooling1D()(char_rnn_1)
    
    char_att_maxpooling_2 = GlobalMaxPooling1D()(char_rnn_2)
    char_att_avgpooling_2 = GlobalAveragePooling1D()(char_rnn_2)
    
#     max_sub_char_abs = Lambda(lambda x:K.abs(x[0] - x[1]))([char_att_maxpooling_1, char_att_maxpooling_2])
#     max_multi_char = Lambda(lambda x: x[0] * x[1])([char_att_maxpooling_1, char_att_maxpooling_2])

#     avg_sub_char_abs = Lambda(lambda x:K.abs(x[0] - x[1]))([char_att_avgpooling_1, char_att_avgpooling_2])
#     avg_multi_char = Lambda(lambda x: x[0] * x[1])([char_att_avgpooling_1, char_att_avgpooling_2])

    merged = concatenate([char_att_maxpooling_1, char_att_avgpooling_1, \
                          char_att_maxpooling_2, char_att_avgpooling_2,])
    merged = Dropout(0.2)(merged)
    merged = BatchNormalization()(merged)
    merged = Dense(512, activation='relu')(merged)
    merged = Dropout(0.1)(merged)
    merged = BatchNormalization()(merged) 
    preds = Dense(19, activation = 'softmax')(merged)
    
    model = Model(inputs=[char_input],outputs=preds)
    return model

In [3]:
meta_train = pd.DataFrame(np.array([None]*train_FE.shape[0]*19).reshape(train_FE.shape[0], 19))
meta_train.columns = ['prob' + str(x) for x in range(19)]
meta_test = pd.DataFrame()

label = train['class']

label = label - 1

categorical_labels = keras.utils.np_utils.to_categorical(label, num_classes=19)

index = pd.read_csv('fold.csv')
index['id'] = train['id']

In [4]:
del train, test, all_word
gc.collect()

48

In [ ]:
%%time
#logloss_list = []
for i in range(10):
    gc.collect()
    K.clear_session()
    print('*******%s*******'%i)
    idx_train = np.array(index[index['fold'] != i]['id'])
    idx_val = np.array(index[index['fold'] == i]['id'])
    
    
    X_train = train_FE.loc[idx_train, :]
    y_train = categorical_labels[idx_train]

    X_val = train_FE.loc[idx_val, :]
    y_val = categorical_labels[idx_val]

    #dtest = test.loc[:,user_col]
    
    X_train = get_input(X_train)
    X_val = get_input(X_val)
    X_test = get_input(test_FE)
    
    BATCH_SIZE = 64
    bst_model_path = '10folds_' + 'base_rnn_wrod_v2_300dim' + str(i) + '.hdf5'
    early_stopping =EarlyStopping(monitor='val_acc', patience=2)
    model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)
    callbacks = [
            early_stopping,
            model_checkpoint
        ]
    model = RNN1()
    model.compile(optimizer='adam', loss='categorical_crossentropy',\
                 metrics=['accuracy'])
    

    hist = model.fit(X_train, y_train, \
            validation_data=(X_val, y_val), \
            epochs=20, batch_size=BATCH_SIZE, shuffle=True, callbacks=callbacks)#callbacks=callbacks
    
    model = RNN1()
    model.load_weights(bst_model_path)
    
    pred = pd.DataFrame(model.predict(X_val,batch_size=64,verbose=1))
    
    for i in range(19):
        meta_train.loc[idx_val, 'prob' + str(i)] = pred[i].values
    
    prob = model.predict(X_test,batch_size=64,verbose=1)
    meta_test = pd.concat([meta_test, pd.DataFrame(prob)], axis = 1)

*******0*******
Train on 92040 samples, validate on 10237 samples
Epoch 1/20
92040/92040 [==============================] - 367s 4ms/step - loss: 1.0371 - acc: 0.7013 - val_loss: 0.9383 - val_acc: 0.7168
Epoch 2/20
92040/92040 [==============================] - 366s 4ms/step - loss: 0.8349 - acc: 0.7521 - val_loss: 0.8795 - val_acc: 0.7418
Epoch 3/20
92040/92040 [==============================] - 366s 4ms/step - loss: 0.7706 - acc: 0.7693 - val_loss: 0.8129 - val_acc: 0.7536
Epoch 4/20
92040/92040 [==============================] - 367s 4ms/step - loss: 0.7201 - acc: 0.7830 - val_loss: 0.8658 - val_acc: 0.7466
Epoch 5/20
92040/92040 [==============================] - 366s 4ms/step - loss: 0.6677 - acc: 0.7967 - val_loss: 0.7720 - val_acc: 0.7672
Epoch 6/20
92040/92040 [==============================] - 366s 4ms/step - loss: 0.6208 - acc: 0.8089 - val_loss: 0.8255 - val_acc: 0.7658
Epoch 7/20
92040/92040 [==============================] - 368s 4ms/step - loss: 0.5699 - acc: 0.8218 - val

In [13]:
%%time
#logloss_list = []
for i in range(10):
    gc.collect()
    K.clear_session()
    print('*******%s*******'%i)
    idx_train = np.array(index[index['fold'] != i]['id'])
    idx_val = np.array(index[index['fold'] == i]['id'])
    
    
    X_train = train_FE.loc[idx_train, :]
    y_train = categorical_labels[idx_train]

    X_val = train_FE.loc[idx_val, :]
    y_val = categorical_labels[idx_val]

    #dtest = test.loc[:,user_col]
    
    X_train = get_input(X_train)
    X_val = get_input(X_val)
    X_test = get_input(test_FE)
    
    BATCH_SIZE = 64
    bst_model_path = '10folds_' + 'base_rnn_wrod_v1_600dim' + str(i) + '.hdf5'
    early_stopping =EarlyStopping(monitor='val_acc', patience=2)
    model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)
    callbacks = [
            early_stopping,
            model_checkpoint
        ]
    model = RNN1()
    model.compile(optimizer='adam', loss='categorical_crossentropy',\
                 metrics=['accuracy'])
    

    hist = model.fit(X_train, y_train, \
            validation_data=(X_val, y_val), \
            epochs=20, batch_size=BATCH_SIZE, shuffle=True, callbacks=callbacks)#callbacks=callbacks
    
    model = RNN1()
    model.load_weights(bst_model_path)
    
    pred = pd.DataFrame(model.predict(X_val,batch_size=64,verbose=1))
    
    for i in range(19):
        meta_train.loc[idx_val, 'prob' + str(i)] = pred[i].values
    
    prob = model.predict(X_test,batch_size=64,verbose=1)
    meta_test = pd.concat([meta_test, pd.DataFrame(prob)], axis = 1)

*******0*******
Train on 92040 samples, validate on 10237 samples
Epoch 1/20
92040/92040 [==============================] - 661s 7ms/step - loss: 1.0202 - acc: 0.7068 - val_loss: 0.8836 - val_acc: 0.7358
Epoch 2/20
92040/92040 [==============================] - 661s 7ms/step - loss: 0.7874 - acc: 0.7650 - val_loss: 0.8301 - val_acc: 0.7525
Epoch 3/20
92040/92040 [==============================] - 661s 7ms/step - loss: 0.6982 - acc: 0.7883 - val_loss: 0.8085 - val_acc: 0.7693
Epoch 4/20
92040/92040 [==============================] - 659s 7ms/step - loss: 0.6064 - acc: 0.8143 - val_loss: 0.8040 - val_acc: 0.7654
Epoch 5/20
92040/92040 [==============================] - 659s 7ms/step - loss: 0.5027 - acc: 0.8404 - val_loss: 0.8374 - val_acc: 0.7667


ResourceExhaustedError: OOM when allocating tensor with shape[1271461,600] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: embedding_2/embeddings/Assign = Assign[T=DT_FLOAT, _class=["loc:@embedding_2/embeddings"], use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](embedding_2/embeddings, embedding_2/random_uniform)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'embedding_2/embeddings/Assign', defined at:
  File "/home/libo/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/libo/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/libo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/libo/anaconda

In [12]:
meta_train.shape

(102277, 19)

In [13]:
meta_test.shape

(102277, 190)

In [14]:
meta_train.columns = ['meta_base_rnn_wrod_v2_300dim_' + x for x in meta_train.columns]

p = pd.DataFrame()

for i in range(19):
    p[i] = meta_test[i].mean(axis = 1)

p.columns = meta_train.columns

In [15]:
p.shape, meta_train.shape

((102277, 19), (102277, 19))

In [16]:
meta_train.to_csv('/home/libo/daguan/stack_new/train_meta_base_rnn_word_v2_300dim.csv', index = None ,encoding = 'utf-8')

p.to_csv('/home/libo/daguan/stack_new/test_meta_base_rnn_word_v2_300dim.csv', index = None ,encoding = 'utf-8')

In [17]:
test = pd.read_csv('test_set.csv')

pred = pd.DataFrame(test['id'])

pred['class'] = np.argmax(p.values, axis=1) + 1

pred.to_csv('/home/libo/daguan/stack_new/9.4.3.csv', index = None ,encoding = 'utf-8')

In [23]:
def RNN():
    
    #Input layer
    char_input = Input(shape=(max_sequence_length_api,), dtype='int32')
    
    api_embedding_layer = Embedding(nb_words,
        embedding_dim,
        weights=[api_embedding_matrix],
        input_length=max_sequence_length_api,
        trainable=False)
    #char_embedding = api_embedding_layer(char_input)
    with tf.device('/cpu:0'):
        char_embedding = api_embedding_layer(char_input)

    
    char_rnn_1= Bidirectional(CuDNNLSTM(100, 
                                return_sequences=True))(char_embedding)
    char_rnn_2= Bidirectional(CuDNNLSTM(100, 
                                return_sequences=True))(char_rnn_1)
    
#     char_rnn_1 = Lambda(lambda x:K.permute_dimensions(x, (0, 2, 1)))(char_rnn_1)
#     char_rnn_2 = Lambda(lambda x:K.permute_dimensions(x, (0, 2, 1)))(char_rnn_2)

    merged = concatenate([char_rnn_1, char_rnn_2], axis = 1)
    
    merged = GlobalAveragePooling1D()(merged)
    #merged = Flatten()(merged)
    #merged = Lambda(lambda x:K.permute_dimensions(x, (0, 2, 1)))(merged)
    
    merged = Dense(1024, activation='relu')(merged)

    preds = Dense(19, activation = 'softmax')(merged)
    
    model = Model(inputs=[char_input],outputs=preds)
    return model

In [24]:
model = RNN()

In [25]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 1000)         0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 1000, 300)    381438300   input_3[0][0]                    
__________________________________________________________________________________________________
bidirectional_5 (Bidirectional) (None, 1000, 200)    321600      embedding_3[0][0]                
__________________________________________________________________________________________________
bidirectional_6 (Bidirectional) (None, 1000, 200)    241600      bidirectional_5[0][0]            
__________________________________________________________________________________________________
concatenat

In [26]:
meta_train = pd.DataFrame(np.array([None]*train_FE.shape[0]*19).reshape(train_FE.shape[0], 19))
meta_train.columns = ['prob' + str(x) for x in range(19)]
meta_test = pd.DataFrame()

label = train['class']

label = label - 1

categorical_labels = keras.utils.np_utils.to_categorical(label, num_classes=19)

index = pd.read_csv('fold.csv')
index['id'] = train['id']

In [27]:
%%time
#logloss_list = []
for i in range(10):
    gc.collect()
    K.clear_session()
    print('*******%s*******'%i)
    idx_train = np.array(index[index['fold'] != i]['id'])
    idx_val = np.array(index[index['fold'] == i]['id'])
    
    
    X_train = train_FE.loc[idx_train, :]
    y_train = categorical_labels[idx_train]

    X_val = train_FE.loc[idx_val, :]
    y_val = categorical_labels[idx_val]

    #dtest = test.loc[:,user_col]
    
    X_train = get_input(X_train)
    X_val = get_input(X_val)
    X_test = get_input(test_FE)
    
    BATCH_SIZE = 64
    bst_model_path = '10folds_' + 'base_rnn_wrod_v2_300dim' + str(i) + '.hdf5'
    early_stopping =EarlyStopping(monitor='val_acc', patience=2)
    model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)
    callbacks = [
            early_stopping,
            model_checkpoint
        ]
    model = RNN()
    model.compile(optimizer='adam', loss='categorical_crossentropy',\
                 metrics=['accuracy'])
    

    hist = model.fit(X_train, y_train, \
            validation_data=(X_val, y_val), \
            epochs=20, batch_size=BATCH_SIZE, shuffle=True, callbacks=callbacks)#callbacks=callbacks
    
    model = RNN()
    model.load_weights(bst_model_path)
    
    pred = pd.DataFrame(model.predict(X_val,batch_size=64,verbose=1))
    
    for i in range(19):
        meta_train.loc[idx_val, 'prob' + str(i)] = pred[i].values
    
    prob = model.predict(X_test,batch_size=64,verbose=1)
    meta_test = pd.concat([meta_test, pd.DataFrame(prob)], axis = 1)

*******0*******
Train on 92040 samples, validate on 10237 samples
Epoch 1/20
92040/92040 [==============================] - 323s 4ms/step - loss: 1.1816 - acc: 0.6528 - val_loss: 0.9639 - val_acc: 0.7230
Epoch 2/20
92040/92040 [==============================] - 321s 3ms/step - loss: 0.8952 - acc: 0.7384 - val_loss: 0.8628 - val_acc: 0.7448
Epoch 3/20
92040/92040 [==============================] - 320s 3ms/step - loss: 0.8149 - acc: 0.7586 - val_loss: 0.8201 - val_acc: 0.7566
Epoch 4/20
92040/92040 [==============================] - 321s 3ms/step - loss: 0.7688 - acc: 0.7714 - val_loss: 0.8114 - val_acc: 0.7576
Epoch 5/20
92040/92040 [==============================] - 321s 3ms/step - loss: 0.7328 - acc: 0.7805 - val_loss: 0.7774 - val_acc: 0.7700
Epoch 6/20
92040/92040 [==============================] - 322s 4ms/step - loss: 0.7024 - acc: 0.7871 - val_loss: 0.7677 - val_acc: 0.7780
Epoch 7/20
92040/92040 [==============================] - 322s 4ms/step - loss: 0.6714 - acc: 0.7962 - val

KeyboardInterrupt: 

In [20]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1000)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1000, 300)    381438300   input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 1000, 200)    321600      embedding_1[0][0]                
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) (None, 1000, 200)    241600      bidirectional_1[0][0]            
__________________________________________________________________________________________________
concatenat